In [1]:
import pandas as pd
import json
from sklearn.feature_selection import SelectKBest, f_classif, chi2
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split as split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Path to the JSON file
file_path = r"C:\Users\MSC\OneDrive - Fraunhofer Austria Research GmbH\Desktop\NLP\data\trans_prob_temp.csv"

# Reading the JSON file into a DataFrame

data = pd.read_csv(file_path)
data.head()  # Display the first few rows of the DataFrame


# Display the first few rows of the dataset
display(data.head())
print(data.info())

,is_sarcastic,headline,pos_tags,syntax_tree,ratio_func_words,ratio_unique_pos,length,pos_repetitions,ratio_noun,ratio_verb,ratio_adj,ratio_adv,ratio_pron,ratio_propn,syntactic_depth,branching_factor,reversed_probability,average_surprisingness
0,1,thirtysomething scientists unveil doomsday clo...,"[{'text': 'thirtysomething', 'pos': 'ADJ', 'de...","[{'text': 'thirtysomething', 'dep': 'amod', 'c...",0.125000,0.625000,8,1,0.500000,0.125000,0.125000,0.000000,0.000,0.125000,5,1.400000,0.289101,0.922017
1,0,dem rep. totally nails why congress is falling...,"[{'text': 'dem', 'pos': 'PROPN', 'dep': 'intj'...","[{'text': 'dem', 'dep': 'intj', 'children': []...",0.266667,0.600000,15,1,0.133333,0.133333,0.133333,0.066667,0.000,0.266667,6,2.800000,0.849506,0.977015
2,0,eat your veggies: 9 deliciously different recipes,"[{'text': 'eat', 'pos': 'VERB', 'dep': 'ROOT',...","[{'text': 'eat', 'dep': 'ROOT', 'children': ['...",0.125000,0.875000,8,0,0.250000,0.125000,0.125000,0.125000,0.125,0.000000,4,1.750000,0.135305,0.887056
3,1,inclement weather prevents liar from getting t...,"[{'text': 'inclement', 'pos': 'NOUN', 'dep': '...","[{'text': 'inclement', 'dep': 'compound', 'chi...",0.250000,0.500000,8,1,0.375000,0.375000,0.000000,0.000000,0.000,0.000000,6,1.166667,0.805611,0.961821
4,1,mother comes pretty close to using word 'strea...,"[{'text': 'mother', 'pos': 'NOUN', 'dep': 'nsu...","[{'text': 'mother', 'dep': 'nsubj', 'children'...",0.272727,0.454545,11,1,0.181818,0.272727,0.000000,0.272727,0.000,0.000000,6,2.000000,0.458575,0.873509


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   is_sarcastic            28619 non-null  int64  
 1   headline                28619 non-null  object 
 2   pos_tags                28619 non-null  object 
 3   syntax_tree             28619 non-null  object 
 4   ratio_func_words        28619 non-null  float64
 5   ratio_unique_pos        28619 non-null  float64
 6   length                  28619 non-null  int64  
 7   pos_repetitions         28619 non-null  int64  
 8   ratio_noun              28619 non-null  float64
 9   ratio_verb              28619 non-null  float64
 10  ratio_adj               28619 non-null  float64
 11  ratio_adv               28619 non-null  float64
 12  ratio_pron              28619 non-null  float64
 13  ratio_propn             28619 non-null  float64
 14  syntactic_depth         28619 non-null

In [3]:
print(data.columns.tolist())


['is_sarcastic', 'headline', 'pos_tags', 'syntax_tree', 'ratio_func_words', 'ratio_unique_pos', 'length', 'pos_repetitions', 'ratio_noun', 'ratio_verb', 'ratio_adj', 'ratio_adv', 'ratio_pron', 'ratio_propn', 'syntactic_depth', 'branching_factor', 'reversed_probability', 'average_surprisingness']


In [4]:
# Load your data (replace `data` with your actual DataFrame)
# Assuming 'data' is already loaded and looks like the provided structure.

# Drop irrelevant columns
data = data.drop(columns=['pos_tags', 'syntax_tree'])

# Define features (X) and target (y)
X = data.iloc[:, 3:]  # Columns 4-17
y = data['is_sarcastic']

# Initialize a dictionary to store feature importance scores
feature_scores = {}

display(X.head())


,ratio_unique_pos,length,pos_repetitions,ratio_noun,ratio_verb,ratio_adj,ratio_adv,ratio_pron,ratio_propn,syntactic_depth,branching_factor,reversed_probability,average_surprisingness
0,0.625000,8,1,0.500000,0.125000,0.125000,0.000000,0.000,0.125000,5,1.400000,0.289101,0.922017
1,0.600000,15,1,0.133333,0.133333,0.133333,0.066667,0.000,0.266667,6,2.800000,0.849506,0.977015
2,0.875000,8,0,0.250000,0.125000,0.125000,0.125000,0.125,0.000000,4,1.750000,0.135305,0.887056
3,0.500000,8,1,0.375000,0.375000,0.000000,0.000000,0.000,0.000000,6,1.166667,0.805611,0.961821
4,0.454545,11,1,0.181818,0.272727,0.000000,0.272727,0.000,0.000000,6,2.000000,0.458575,0.873509


In [5]:
# 1. Filter Method: Univariate Statistical Test (ANOVA F-test)
select_k_best = SelectKBest(score_func=f_classif, k='all')
anova_scores = select_k_best.fit(X, y).scores_

# 2. Wrapper Method: Recursive Feature Elimination (RFE) with Logistic Regression
log_model = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=log_model, n_features_to_select=5)  # Select top 5 features
rfe.fit(X, y)
rfe_ranks = rfe.ranking_

# 3. Embedded Method: Feature Importance from Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X, y)
rf_importances = rf_model.feature_importances_

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale each method's scores to [0, 1]
anova_scaled = scaler.fit_transform(anova_scores.reshape(-1, 1)).flatten()
rfe_scaled = scaler.fit_transform((1 / rfe_ranks).reshape(-1, 1)).flatten()  # Inverse for ranking
rf_scaled = scaler.fit_transform(rf_importances.reshape(-1, 1)).flatten()

# Combine the scores into a DataFrame
feature_scores = {
    'ANOVA_F': anova_scaled,
    'RFE': rfe_scaled,
    'RandomForest': rf_scaled
}
importance_df = pd.DataFrame(feature_scores, index=X.columns)

# Add an average score column
importance_df['Average_Score'] = importance_df.mean(axis=1)
importance_df = importance_df.sort_values(by='Average_Score', ascending=False)

print("Feature Importance Scores:")
print(importance_df)

# Select features based on average score threshold
selected_features = importance_df[importance_df['Average_Score'] > 0.05].index.tolist()
print("Selected Features:", selected_features)

# # Keep only selected features
# X_selected = X[selected_features]

# # Updated dataset with selected features and headlines
# data_transformed = pd.concat([data[['is_sarcastic', 'headline']], X_selected], axis=1)
# print("Transformed DataFrame:")
# print(data_transformed.head())


Feature Importance Scores:
                         ANOVA_F       RFE  RandomForest  Average_Score
ratio_verb              1.000000  1.000000      0.619594       0.873198
ratio_unique_pos        0.655548  1.000000      0.536356       0.730634
ratio_noun              0.526055  1.000000      0.500238       0.675431
ratio_adv               0.496160  1.000000      0.262436       0.586199
ratio_adj               0.187332  1.000000      0.358383       0.515238
branching_factor        0.761033  0.156250      0.518813       0.478699
average_surprisingness  0.062645  0.250000      1.000000       0.437548
reversed_probability    0.000000  0.100000      0.979131       0.359710
syntactic_depth         0.513614  0.035714      0.281690       0.277006
ratio_pron              0.255375  0.437500      0.135734       0.276203
ratio_propn             0.262081  0.062500      0.354240       0.226274
length                  0.021638  0.000000      0.199567       0.073735
pos_repetitions         0.015240  0.0

In [6]:


# 2. Wrapper Method: Recursive Feature Elimination (RFE) with Logistic Regression
log_model = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=log_model, n_features_to_select=5)  # Select top 5 features
rfe.fit(X, y)
feature_scores['RFE'] = rfe.ranking_

# 3. Embedded Method: Feature Importance from Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X, y)
feature_scores['RandomForest'] = rf_model.feature_importances_

# Combine results into a DataFrame for comparison
importance_df = pd.DataFrame(feature_scores, index=X.columns)
importance_df['Average_Score'] = importance_df.mean(axis=1)
importance_df = importance_df.sort_values(by='Average_Score', ascending=False)

print("Feature Importance Scores:")
print(importance_df)

# Select features based on average score threshold
selected_features = importance_df[importance_df['Average_Score'] > 0.05].index.tolist()
print("Selected Features:", selected_features)

# Keep only selected features
X_selected = X[selected_features]

# Updated dataset with selected features and headlines
data_transformed = pd.concat([data[['is_sarcastic', 'headline']], X_selected], axis=1)
print("Transformed DataFrame:")
display(data_transformed.head())


Feature Importance Scores:
                         ANOVA_F  RFE  RandomForest  Average_Score
length                  0.021638    9      0.045433       3.022357
pos_repetitions         0.015240    8      0.018865       2.678035
syntactic_depth         0.513614    7      0.055389       2.523001
ratio_propn             0.262081    6      0.065784       2.109288
reversed_probability    0.000000    5      0.148160       1.716053
branching_factor        0.761033    4      0.086825       1.615952
average_surprisingness  0.062645    3      0.151010       1.071219
ratio_pron              0.255375    2      0.036807       0.764061
ratio_verb              1.000000    1      0.099849       0.699950
ratio_unique_pos        0.655548    1      0.089075       0.581541
ratio_noun              0.526055    1      0.083633       0.536563
ratio_adv               0.496160    1      0.052452       0.516204
ratio_adj               0.187332    1      0.066720       0.418017
Selected Features: ['length', 'pos_

,is_sarcastic,headline,length,pos_repetitions,syntactic_depth,ratio_propn,reversed_probability,branching_factor,average_surprisingness,ratio_pron,ratio_verb,ratio_unique_pos,ratio_noun,ratio_adv,ratio_adj
0,1,thirtysomething scientists unveil doomsday clo...,8,1,5,0.125000,0.289101,1.400000,0.922017,0.000,0.125000,0.625000,0.500000,0.000000,0.125000
1,0,dem rep. totally nails why congress is falling...,15,1,6,0.266667,0.849506,2.800000,0.977015,0.000,0.133333,0.600000,0.133333,0.066667,0.133333
2,0,eat your veggies: 9 deliciously different recipes,8,0,4,0.000000,0.135305,1.750000,0.887056,0.125,0.125000,0.875000,0.250000,0.125000,0.125000
3,1,inclement weather prevents liar from getting t...,8,1,6,0.000000,0.805611,1.166667,0.961821,0.000,0.375000,0.500000,0.375000,0.000000,0.000000
4,1,mother comes pretty close to using word 'strea...,11,1,6,0.000000,0.458575,2.000000,0.873509,0.000,0.272727,0.454545,0.181818,0.272727,0.000000


In [7]:
# Split the data into training, validation, and test sets
SEED = 42
train_headlines, other_headlines = split(data, test_size=0.3, random_state=SEED)  # 70% training
val_headlines, test_headlines = split(other_headlines, test_size=0.5, random_state=SEED)  # 15% validation, 15% test

# Prepare data for the Logistic Regression model
# Extract the corresponding feature values for train, validation, and test
X_train = X.loc[train_headlines.index]
y_train = y.loc[train_headlines.index]
X_val = X.loc[val_headlines.index]
y_val = y.loc[val_headlines.index]
X_test = X.loc[test_headlines.index]
y_test = y.loc[test_headlines.index]

In [8]:
# Select the 5 best features from RFE scores
top_5_features = importance_df.sort_values(by='RFE').head(5).index.tolist()
print(top_5_features)

# Use only the top 5 features for training and testing
X_train_top5 = X_train[top_5_features]
X_test_top5 = X_test[top_5_features]

# Train the Logistic Regression model
log_model = LogisticRegression(max_iter=1000, random_state=SEED)
log_model.fit(X_train_top5, y_train)

# Make predictions on the test set
y_pred = log_model.predict(X_test_top5)

# Print confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

['ratio_verb', 'ratio_unique_pos', 'ratio_noun', 'ratio_adv', 'ratio_adj']
Confusion Matrix:
[[1632  605]
 [ 747 1309]]

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.73      0.71      2237
           1       0.68      0.64      0.66      2056

    accuracy                           0.69      4293
   macro avg       0.68      0.68      0.68      4293
weighted avg       0.68      0.69      0.68      4293



In [9]:
# Select the 5 best features from Random Forest importance scores
top_5_rf_features = importance_df.sort_values(by='RandomForest', ascending=False).head(5).index.tolist()
print(top_5_rf_features)

# Use only the top 5 features for training and testing
X_train_top5_rf = X_train[top_5_rf_features]
X_test_top5_rf = X_test[top_5_rf_features]

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=SEED)
rf_model.fit(X_train_top5_rf, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test_top5_rf)

# Print confusion matrix and classification report
print("Confusion Matrix (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf))

['average_surprisingness', 'reversed_probability', 'ratio_verb', 'ratio_unique_pos', 'branching_factor']
Confusion Matrix (Random Forest):
[[1445  792]
 [ 809 1247]]

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.64      0.65      0.64      2237
           1       0.61      0.61      0.61      2056

    accuracy                           0.63      4293
   macro avg       0.63      0.63      0.63      4293
weighted avg       0.63      0.63      0.63      4293



# Ensembles

In [10]:
# Path to the JSON file
file_path = r"C:\Users\MSC\OneDrive - Fraunhofer Austria Research GmbH\Desktop\NLP\data\trans_prob_temp.csv"

# Reading the JSON file into a DataFrame

data = pd.read_csv(file_path)
data.head()  # Display the first few rows of the DataFrame

top_5_features.append('headline')
print(top_5_features)
X = data[top_5_features]

['ratio_verb', 'ratio_unique_pos', 'ratio_noun', 'ratio_adv', 'ratio_adj', 'headline']


In [11]:
display(X.head())

,ratio_verb,ratio_unique_pos,ratio_noun,ratio_adv,ratio_adj,headline
0,0.125000,0.625000,0.500000,0.000000,0.125000,thirtysomething scientists unveil doomsday clo...
1,0.133333,0.600000,0.133333,0.066667,0.133333,dem rep. totally nails why congress is falling...
2,0.125000,0.875000,0.250000,0.125000,0.125000,eat your veggies: 9 deliciously different recipes
3,0.375000,0.500000,0.375000,0.000000,0.000000,inclement weather prevents liar from getting t...
4,0.272727,0.454545,0.181818,0.272727,0.000000,mother comes pretty close to using word 'strea...
